# Kepler Framework, Emulate v11

In [1]:
from kepler.pandas.menu       import ElectronSequence as Chain
from kepler.pandas.readers    import load, load_in_loop, load_hdf
from core import Efficiency
from Gaugi import mkdir_p, progressbar
import numpy as np
import pandas as pd
import collections
import os
from pprint import pprint
from copy import deepcopy

import gc
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

Welcome to JupyROOT 6.16/00
Using all sub packages with ROOT dependence
INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.

Applying ATLAS style settings...

Applying ATLAS style settings...

Applying ATLAS style settings...


In [2]:
output_path = 'output/emulation'
mkdir_p(output_path)

In [3]:
dpath_data17 = '../../data/output/data17_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.dataframe.h5'
dpath_data18 = '../../data/output/data18_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.dataframe.h5'
dpath_mc16   = '../../data/output/mc16_13TeV.302236_309995_341330.sgn.boosted_probes.WZ_llqq_plus_radion_ZZ_llqq_plus_ggH3000.merge.dataframe.h5'

In [4]:
tpath_data17 = 'output/emulation/data17_table_v11.h5'
tpath_data18 = 'output/emulation/data18_table_v11.h5'
tpath_mc16   = 'output/emulation/mc16_table_v11.h5'

## Setup Chains:

In [5]:
# create my chain
chains = [
            Chain( "HLT_e17_lhvloose_nod0_ringer_v11_L1EM15VHI", L1Seed = 'L1_EM15VHI' , l2calo_column = 'ringer_v11_vloose'),
            Chain( "HLT_e28_lhtight_nod0_ringer_v11_ivarloose", L1Seed = 'L1_EM22VHI' , l2calo_column = 'ringer_v11_tight' ),
            Chain( "HLT_e60_lhmedium_nod0_ringer_v11_L1EM24VHI", L1Seed = 'L1_EM24VHI' , l2calo_column = 'ringer_v11_medium'),
            Chain( "HLT_e140_lhloose_nod0_ringer_v11" , L1Seed = 'L1_EM24VHI', l2calo_column = 'ringer_v11_loose'),
]

def emulate( df ):
    for chain in progressbar(chains, prefix='Emulate...'):
        chain.apply(df)

OrderedDict([('L1Seed', 'L1_EM15VHI'),
             ('L2Calo', (14000.0, 'ringer_v11_vloose')),
             ('L2', 'trig_L2_el_cut_pt15to20'),
             ('EFCalo', 17000.0),
             ('HLT', (17000.0, 'trig_EF_el_lhvloose'))])
OrderedDict([('L1Seed', 'L1_EM22VHI'),
             ('L2Calo', (25000.0, 'ringer_v11_tight')),
             ('L2', 'trig_L2_el_cut_pt20to50'),
             ('EFCalo', 28000.0),
             ('HLT', (28000.0, 'trig_EF_el_lhtight_ivarloose'))])
OrderedDict([('L1Seed', 'L1_EM24VHI'),
             ('L2Calo', (57000.0, 'ringer_v11_medium')),
             ('L2', 'trig_L2_el_cut_pt50toInf'),
             ('EFCalo', 60000.0),
             ('HLT', (60000.0, 'trig_EF_el_lhmedium'))])
OrderedDict([('L1Seed', 'L1_EM24VHI'),
             ('L2Calo', (137000.0, 'ringer_v11_loose')),
             ('L2', 'trig_L2_el_cut_pt50toInf'),
             ('EFCalo', 140000.0),
             ('HLT', (140000.0, 'trig_EF_el_lhloose'))])


## Emulate 2017:

Take something like 1 hour

In [6]:
data17_df = pd.concat( (load_hdf(dpath_data17), load_hdf(tpath_data17)), axis=1)

In [7]:
emulate(data17_df)

Emulate...: 100%|██████████| 4/4 [01:25<00:00, 21.47s/it]


In [8]:
data17_df.shape

(43311283, 111)

### Validate:

In [9]:
eff_data17_Zee = Efficiency( output_path+'/data17_Zee_efficiency_v11.root')

2022-03-13 10:27:44,999 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/data17_Zee_efficiency_v11.root


In [10]:
eff_data17_Zee.fill( data17_df.loc[data17_df['target']==1], 'HLT_e17_lhvloose_nod0_ringer_v11_L1EM15VHI', pidname='el_lhvloose', reset=True)
eff_data17_Zee.fill( data17_df.loc[data17_df['target']==1], 'HLT_e28_lhtight_nod0_ringer_v11_ivarloose' , pidname='el_lhtight' , reset=True)
eff_data17_Zee.fill( data17_df.loc[data17_df['target']==1], 'HLT_e60_lhmedium_nod0_ringer_v11_L1EM24VHI', pidname='el_lhmedium', reset=True)
eff_data17_Zee.fill( data17_df.loc[data17_df['target']==1], 'HLT_e140_lhloose_nod0_ringer_v11'          , pidname='el_lhloose' , reset=True)

Filling...: 100%|██████████| 5/5 [01:52<00:00, 22.47s/it]


HLT_e17_lhvloose_nod0_ringer_v11_L1EM15VHI
+--------+----------+-------------------+
|  Step  | Eff [%%] |    passed/total   |
+--------+----------+-------------------+
| L1Calo | 94.1069  | 30158981/32047563 |
| L2Calo | 93.4437  | 29946426/32047563 |
|   L2   | 93.2030  | 29869275/32047563 |
| EFCalo | 91.8928  | 29449398/32047563 |
|  HLT   | 89.9885  | 28839114/32047563 |
+--------+----------+-------------------+


Filling...: 100%|██████████| 5/5 [01:21<00:00, 16.30s/it]


HLT_e28_lhtight_nod0_ringer_v11_ivarloose
+--------+----------+-------------------+
|  Step  | Eff [%%] |    passed/total   |
+--------+----------+-------------------+
| L1Calo | 94.6937  | 21557594/22765602 |
| L2Calo | 94.2276  | 21451488/22765602 |
|   L2   | 94.1117  | 21425084/22765602 |
| EFCalo | 93.1861  | 21214387/22765602 |
|  HLT   | 87.0066  | 19807570/22765602 |
+--------+----------+-------------------+


Filling...: 100%|██████████| 5/5 [00:05<00:00,  1.18s/it]


HLT_e60_lhmedium_nod0_ringer_v11_L1EM24VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.5079  | 1361055/1367786 |
| L2Calo | 96.7072  | 1322747/1367786 |
|   L2   | 96.6160  | 1321500/1367786 |
| EFCalo | 94.5202  | 1292834/1367786 |
|  HLT   | 91.2530  | 1248146/1367786 |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:00<00:00, 18.71it/s]

HLT_e140_lhloose_nod0_ringer_v11
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 100.0000 | 60541/60541  |
| L2Calo | 95.0298  | 57532/60541  |
|   L2   | 94.9654  | 57493/60541  |
| EFCalo | 93.6886  | 56720/60541  |
|  HLT   | 92.0467  | 55726/60541  |
+--------+----------+--------------+


In [11]:
eff_data17_Zee.save()

In [12]:
eff_data17_JF17 = Efficiency( output_path+'/data17_JF17_efficiency_v11.root')

2022-03-13 10:31:50,986 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/data17_JF17_efficiency_v11.root


In [13]:
eff_data17_JF17.fill( data17_df.loc[data17_df['target']==0], 'HLT_e17_lhvloose_nod0_ringer_v11_L1EM15VHI', reset=True)
eff_data17_JF17.fill( data17_df.loc[data17_df['target']==0], 'HLT_e28_lhtight_nod0_ringer_v11_ivarloose' , reset=True)
eff_data17_JF17.fill( data17_df.loc[data17_df['target']==0], 'HLT_e60_lhmedium_nod0_ringer_v11_L1EM24VHI', reset=True)
eff_data17_JF17.fill( data17_df.loc[data17_df['target']==0], 'HLT_e140_lhloose_nod0_ringer_v11'          , reset=True)

Filling...: 100%|██████████| 5/5 [00:22<00:00,  4.44s/it]


HLT_e17_lhvloose_nod0_ringer_v11_L1EM15VHI
+--------+----------+------------------+
|  Step  | Eff [%%] |   passed/total   |
+--------+----------+------------------+
| L1Calo | 84.4275  | 8738241/10349992 |
| L2Calo |  0.7103  |  73520/10349992  |
|   L2   |  0.6711  |  69455/10349992  |
| EFCalo |  0.5852  |  60569/10349992  |
|  HLT   |  0.0682  |  7057/10349992   |
+--------+----------+------------------+


Filling...: 100%|██████████| 5/5 [00:19<00:00,  3.91s/it]


HLT_e28_lhtight_nod0_ringer_v11_ivarloose
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 90.6060  | 8384948/9254300 |
| L2Calo |  0.2340  |  21655/9254300  |
|   L2   |  0.2142  |  19824/9254300  |
| EFCalo |  0.1784  |  16506/9254300  |
|  HLT   |  0.0008  |    72/9254300   |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:16<00:00,  3.30s/it]


HLT_e60_lhmedium_nod0_ringer_v11_L1EM24VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.9012  | 7844422/7852181 |
| L2Calo |  0.1441  |  11316/7852181  |
|   L2   |  0.1198  |   9410/7852181  |
| EFCalo |  0.0757  |   5944/7852181  |
|  HLT   |  0.0004  |    33/7852181   |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:12<00:00,  2.44s/it]

HLT_e140_lhloose_nod0_ringer_v11
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.9227  | 5629684/5634041 |
| L2Calo |  0.3425  |  19297/5634041  |
|   L2   |  0.2918  |  16442/5634041  |
| EFCalo |  0.1904  |  10727/5634041  |
|  HLT   |  0.0093  |   522/5634041   |
+--------+----------+-----------------+


In [14]:
eff_data17_JF17.save()

In [15]:
del data17_df

## Emulate 2018:

In [16]:
data18_df = pd.concat( (load_hdf(dpath_data18), load_hdf(tpath_data18)), axis=1)

In [17]:
emulate(data18_df)

Emulate...: 100%|██████████| 4/4 [01:21<00:00, 20.28s/it]


In [18]:
data18_df.shape

(41873973, 111)

In [19]:
eff_data18_Zee = Efficiency( output_path+'/data18_Zee_efficiency_v11.root')

2022-03-13 10:35:42,163 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/data18_Zee_efficiency_v11.root


In [20]:
eff_data18_Zee.fill( data18_df.loc[data18_df['target']==1], 'HLT_e17_lhvloose_nod0_ringer_v11_L1EM15VHI', pidname='el_lhvloose', reset=True)
eff_data18_Zee.fill( data18_df.loc[data18_df['target']==1], 'HLT_e28_lhtight_nod0_ringer_v11_ivarloose' , pidname='el_lhtight' , reset=True)
eff_data18_Zee.fill( data18_df.loc[data18_df['target']==1], 'HLT_e60_lhmedium_nod0_ringer_v11_L1EM24VHI', pidname='el_lhmedium', reset=True)
eff_data18_Zee.fill( data18_df.loc[data18_df['target']==1], 'HLT_e140_lhloose_nod0_ringer_v11'          , pidname='el_lhloose' , reset=True)

Filling...: 100%|██████████| 5/5 [01:51<00:00, 22.36s/it]


HLT_e17_lhvloose_nod0_ringer_v11_L1EM15VHI
+--------+----------+-------------------+
|  Step  | Eff [%%] |    passed/total   |
+--------+----------+-------------------+
| L1Calo | 94.1301  | 30705693/32620495 |
| L2Calo | 93.4830  | 30494610/32620495 |
|   L2   | 93.2457  | 30417199/32620495 |
| EFCalo | 91.9131  | 29982516/32620495 |
|  HLT   | 90.4885  | 29517802/32620495 |
+--------+----------+-------------------+


Filling...: 100%|██████████| 5/5 [01:21<00:00, 16.38s/it]


HLT_e28_lhtight_nod0_ringer_v11_ivarloose
+--------+----------+-------------------+
|  Step  | Eff [%%] |    passed/total   |
+--------+----------+-------------------+
| L1Calo | 94.6555  | 22067825/23313826 |
| L2Calo | 94.1928  | 21959941/23313826 |
|   L2   | 94.0965  | 21937494/23313826 |
| EFCalo | 93.2964  | 21750958/23313826 |
|  HLT   | 87.5740  | 20416853/23313826 |
+--------+----------+-------------------+


Filling...: 100%|██████████| 5/5 [00:05<00:00,  1.18s/it]


HLT_e60_lhmedium_nod0_ringer_v11_L1EM24VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.5563  | 1375545/1381675 |
| L2Calo | 96.7214  | 1336376/1381675 |
|   L2   | 96.6312  | 1335129/1381675 |
| EFCalo | 94.5389  | 1306221/1381675 |
|  HLT   | 91.5744  | 1265261/1381675 |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:00<00:00, 20.57it/s]

HLT_e140_lhloose_nod0_ringer_v11
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 99.9934  | 60385/60389  |
| L2Calo | 94.8269  | 57265/60389  |
|   L2   | 94.7524  | 57220/60389  |
| EFCalo | 93.6164  | 56534/60389  |
|  HLT   | 92.1045  | 55621/60389  |
+--------+----------+--------------+


In [21]:
eff_data18_Zee.save()

In [22]:
eff_data18_JF17 = Efficiency( output_path+'/data18_JF17_efficiency_v11.root')

2022-03-13 10:39:48,078 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/data18_JF17_efficiency_v11.root


In [23]:
eff_data18_JF17.fill( data18_df.loc[data18_df['target']==0], 'HLT_e17_lhvloose_nod0_ringer_v11_L1EM15VHI', reset=True)
eff_data18_JF17.fill( data18_df.loc[data18_df['target']==0], 'HLT_e28_lhtight_nod0_ringer_v11_ivarloose' , reset=True)
eff_data18_JF17.fill( data18_df.loc[data18_df['target']==0], 'HLT_e60_lhmedium_nod0_ringer_v11_L1EM24VHI', reset=True)
eff_data18_JF17.fill( data18_df.loc[data18_df['target']==0], 'HLT_e140_lhloose_nod0_ringer_v11'          , reset=True)

Filling...: 100%|██████████| 5/5 [00:17<00:00,  3.48s/it]


HLT_e17_lhvloose_nod0_ringer_v11_L1EM15VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 85.5183  | 7240922/8467098 |
| L2Calo |  1.0161  |  86037/8467098  |
|   L2   |  0.9788  |  82874/8467098  |
| EFCalo |  0.8936  |  75664/8467098  |
|  HLT   |  0.3769  |  31912/8467098  |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:15<00:00,  3.16s/it]


HLT_e28_lhtight_nod0_ringer_v11_ivarloose
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 91.4008  | 6972786/7628804 |
| L2Calo |  0.4471  |  34107/7628804  |
|   L2   |  0.4280  |  32649/7628804  |
| EFCalo |  0.3912  |  29843/7628804  |
|  HLT   |  0.0244  |   1864/7628804  |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:13<00:00,  2.70s/it]


HLT_e60_lhmedium_nod0_ringer_v11_L1EM24VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.8995  | 6555389/6561985 |
| L2Calo |  0.3074  |  20173/6561985  |
|   L2   |  0.2838  |  18626/6561985  |
| EFCalo |  0.2400  |  15746/6561985  |
|  HLT   |  0.0262  |   1717/6561985  |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:10<00:00,  2.02s/it]

HLT_e140_lhloose_nod0_ringer_v11
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.9159  | 4772502/4776518 |
| L2Calo |  0.6175  |  29496/4776518  |
|   L2   |  0.5685  |  27155/4776518  |
| EFCalo |  0.4687  |  22388/4776518  |
|  HLT   |  0.1925  |   9196/4776518  |
+--------+----------+-----------------+


In [24]:
eff_data18_JF17.save()

## Evaluate mc16 boosted:

In [25]:
mc16_df = pd.concat( (load_hdf(dpath_mc16), load_hdf(tpath_mc16)), axis=1)

In [26]:
emulate(mc16_df)

Emulate...: 100%|██████████| 4/4 [00:00<00:00, 42.50it/s]


In [27]:
eff_mc16 = Efficiency( output_path+'/mc16_boosted_efficiency_v11.root')

2022-03-13 10:41:00,716 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/mc16_boosted_efficiency_v11.root


In [28]:
eff_mc16.fill( mc16_df.loc[mc16_df['target']==1], 'HLT_e17_lhvloose_nod0_ringer_v11_L1EM15VHI', pidname='el_lhvloose', reset=True)
eff_mc16.fill( mc16_df.loc[mc16_df['target']==1], 'HLT_e28_lhtight_nod0_ringer_v11_ivarloose' , pidname='el_lhtight' , reset=True)
eff_mc16.fill( mc16_df.loc[mc16_df['target']==1], 'HLT_e60_lhmedium_nod0_ringer_v11_L1EM24VHI', pidname='el_lhmedium', reset=True)
eff_mc16.fill( mc16_df.loc[mc16_df['target']==1], 'HLT_e140_lhloose_nod0_ringer_v11'          , pidname='el_lhloose' , reset=True)

Filling...: 100%|██████████| 5/5 [00:00<00:00, 17.47it/s]


HLT_e17_lhvloose_nod0_ringer_v11_L1EM15VHI
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 99.7303  | 78407/78619  |
| L2Calo | 74.0801  | 58241/78619  |
|   L2   | 74.0699  | 58233/78619  |
| EFCalo | 60.7334  | 47748/78619  |
|  HLT   | 59.8647  | 47065/78619  |
+--------+----------+--------------+


Filling...: 100%|██████████| 5/5 [00:00<00:00, 21.96it/s]


HLT_e28_lhtight_nod0_ringer_v11_ivarloose
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 99.8427  | 72990/73105  |
| L2Calo | 35.2835  | 25794/73105  |
|   L2   | 35.2780  | 25790/73105  |
| EFCalo | 30.9158  | 22601/73105  |
|  HLT   | 29.6491  | 21675/73105  |
+--------+----------+--------------+


Filling...: 100%|██████████| 5/5 [00:00<00:00, 21.32it/s]


HLT_e60_lhmedium_nod0_ringer_v11_L1EM24VHI
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 99.9635  | 73911/73938  |
| L2Calo | 39.1964  | 28981/73938  |
|   L2   | 39.1909  | 28977/73938  |
| EFCalo | 33.8121  | 25000/73938  |
|  HLT   | 32.9087  | 24332/73938  |
+--------+----------+--------------+


Filling...: 100%|██████████| 5/5 [00:00<00:00, 20.18it/s]

HLT_e140_lhloose_nod0_ringer_v11
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 99.9882  | 67947/67955  |
| L2Calo | 68.9206  | 46835/67955  |
|   L2   | 68.9162  | 46832/67955  |
| EFCalo | 56.0959  | 38120/67955  |
|  HLT   | 55.0820  | 37431/67955  |
+--------+----------+--------------+


In [29]:
eff_mc16.save()